# Simulating the nucleosynthesis of elements in stars

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from scipy import integrate

Construct a series of differential equations to simulate the nucleosynthesis of elements in stars due to the CNO1 cycle.

The differential equations are given by:

\begin{align}
\frac{d C12}{dt} &= H \left(N15 \right) \langle \sigma v \rangle_{N15 (p,\alpha)} - H \left(C12 \right) \langle \sigma v \rangle_{C12 (p,\gamma)} \\
\frac{d C13}{dt} &= H \left(C12 \right) \langle \sigma v \rangle_{C12 (p,\gamma)} - H \left(C13 \right) \langle \sigma v \rangle_{C13 (p,\gamma)} \\
\frac{d N14}{dt} &= H \left(C13 \right) \langle \sigma v \rangle_{C13 (p,\gamma)} - H \left(N14 \right) \langle \sigma v \rangle_{N14 (p,\gamma)} \\
\frac{d N15}{dt} &= H \left(N14 \right) \langle \sigma v \rangle_{N14 (p,\gamma)} - H \left(N15 \right) \langle \sigma v \rangle_{N15 (p,\alpha)} \\
\end{align}

Where:
- $H(X)$ is the relative abundance of element $X$.
- $\langle \sigma v \rangle_{X (p,\gamma/\alpha)}$ is the reaction rate of the reaction $X (p,\gamma/\alpha)$ determined by the following equation:

\begin{align}
\langle \sigma v \rangle_{X (p,\gamma/\alpha)} &= \int_0^{\infty} v \sigma(v) P(v) dv \\
&= \left( \frac{8}{\pi m_{01}}\right)^{1/2} \frac{1}{(kT)^{3/2}} \int_0^{\infty} E \sigma(E) e^{-E/kT} dE
\end{align}

Where:
- $m_{01}$ is the reduced mass of the system.
- $k$ is the Boltzmann constant.
- $T$ is the temperature of the star's core.
- $E$ is the energy
- $\sigma(E)$ is the cross section of the reaction.

Note: We ignore the reactions involving $O16$ and $N13$ because they decay very quickly relative to the other elements.

In [ ]:
def reaction_rate(temp:float, mass:float, crossSection:float, kB = 8.617333262e-5)->float:
    """
    Calculate the reaction rate (sigma v) for the respective reaction.
    
    Parameters
    ----------
    
    temp : float
        The temperature of the system in Kelvin
    mass : float
        The reduced mass of the system
    crossSection : float
        The cross-section of the reaction
    kB : float
        The Boltzmann constant. Default is 8.617333262e-5 in eV/K
    
    Returns
    -------
    
    rate : float
        The reaction rate
    """
    constant = (8/(np.pi*mass))**(1/2) * 1/(kB * temp)**(3/2)
    
    integral = integrate.quad(
        lambda E: E * np.exp(-E/(kB * temp))*crossSection,
        0, np.inf)
    
    return constant * integral[0]

In [ ]:
def calc_red_mass(m1:float, m2:float)->float:
    """
    Calculate the reduced mass of a system
    
    Parameters
    ----------
    
    m1 : float
        The mass of the first particle
    m2 : float
        The mass of the second particle
    
    Returns
    -------
    
    red_mass : float
        The reduced mass of the system
    """
    return m1*m2/(m1+m2)

In [ ]:
def CNO1_cycle(elements:dict, masses:dict, temp=15e6, crossSection = 1)->dict:
    """
    Construct the differential equations for systems of the CNO1 cycle.
    
    Parameters
    ----------
    elements : dict
        Dictionary containing the current values of the initialAbundances.
    masses : dict
        Masses of the initialAbundances in the system.
    temp : float
        The temperature of the system in Kelvin. Default is 15 million Kelvin.
    crossSection : float
        The cross-section of the reaction. Default is 1 for the time being.
        
    Returns
    -------
    dX : dict
        The differential equations for the system.
    """
    
    rates = {
        'C12toC13': reaction_rate(temp, calc_red_mass(masses['C12'], masses['H1']), crossSection) * elements['C12']/elements['H1'],
        'C13toN14': reaction_rate(temp, calc_red_mass(masses['C13'], masses['H1']), crossSection) * elements['C13']/elements['H1'],
        'N14toN15': reaction_rate(temp, calc_red_mass(masses['N14'], masses['H1']), crossSection) * elements['N14']/elements['H1'],
        'N15toC12': reaction_rate(temp, calc_red_mass(masses['N15'], masses['H1']), crossSection) * elements['N15']/elements['H1']
        }
    
    dC12 = rates['N15toC12'] - rates['C12toC13']
    dC13 = rates['C12toC13'] - rates['C13toN14']
    dN14 = rates['C13toN14'] - rates['N14toN15']
    dN15 = rates['N14toN15'] - rates['N15toC12']
    dH1 = -rates['N15toC12'] - rates['C12toC13'] - rates['C13toN14'] - rates['N14toN15']
    dHe4 = rates['N15toC12']
    
    
    dX = {'C12': dC12, 'C13': dC13, 'N14': dN14, 'N15': dN15, 'H1': dH1, 'He4': dHe4}

    return dX

The CNO2 cycle is a series of reactions that occur in stars that are more massive than the Sun. The CNO2 cycle is similar to the CNO1 cycle, but it involves the elements $N14$, $N15$, $O15$, $O16$, $O17$, and $F17$.

Due to the larger masses of the elements involved in the CNO2 cycle, these reactions are typically less likely to occur than the reactions in the CNO1 cycle. We expect this cycle to therefore have a smaller impact on the abundance of elements in the star's core.

The differential equations for the CNO2 cycle are given by:

\begin{align}

\frac{d N14}{dt} &= H \left(O17 \right) \langle \sigma v \rangle_{O17 (p,\alpha)} - H \left(N14 \right) \langle \sigma v \rangle_{N14 (p,\gamma)} \\
\frac{d N15}{dt} &= H \left(N14 \right) \langle \sigma v \rangle_{N14 (p,\gamma)} - H \left(N15 \right) \langle \sigma v \rangle_{N15 (p,\gamma)} \\
\frac{d O16}{dt} &= H \left(N15 \right) \langle \sigma v \rangle_{N15 (p,\gamma)} - H \left(O16 \right) \langle \sigma v \rangle_{O16 (p,\gamma)} \\
\frac{d O17}{dt} &= H \left(O16 \right) \langle \sigma v \rangle_{O16 (p,\gamma)} - H \left(O17 \right) \langle \sigma v \rangle_{O17 (p,\alpha)} \\
\end{align}


In [ ]:
def CNO2_cycle(elements:dict, masses:dict, temp=15e6, crossSection = 1)->dict:
    """
    Construct the differential equations for systems of the CNO2 cycle.

    Parameters
    ----------
    elements : dict
        Dictionary containing the current values of the initialAbundances.
    masses : dict
        Masses of the initialAbundances in the system.
    temp : float
        The temperature of the system in Kelvin. Default is 15 million Kelvin.
    crossSection : float
        The cross-section of the reaction. Default is 1 for the time being.

    Returns
    -------
    dX : dict
        The differential equations for the system.
    """
    
    rates ={
        'N14toN15': reaction_rate(temp, calc_red_mass(masses['N14'], masses['H1']), crossSection) * elements['N14']/elements['H1'],
        'N15toO16': reaction_rate(temp, calc_red_mass(masses['N15'], masses['H1']), crossSection) * elements['N15']/elements['H1'],
        'O16toO17': reaction_rate(temp, calc_red_mass(masses['O16'], masses['H1']), crossSection) * elements['O16']/elements['H1'],
        'O17toN14': reaction_rate(temp, calc_red_mass(masses['O17'], masses['H1']), crossSection) * elements['O17']/elements['H1']
    }
    

    dN15 = rates['N14toN15'] - rates['N15toO16']
    dO16 = rates['N15toO16'] - rates['O16toO17']
    dO17 = rates['O16toO17'] - rates['O17toN14']
    dN14 = rates['O17toN14'] - rates['N14toN15']
    dH1 = -rates['N14toN15'] - rates['N15toO16'] - rates['O16toO17'] - rates['O17toN14']
    dHe4 = rates['O17toN14']
    
    dX = {'N14': dN14, 'N15': dN15, 'O16': dO16, 'O17': dO17, 'H1': dH1, 'He4': dHe4}

    return dX

In [ ]:
from hmac import new


def integrate_system(elements:dict, masses:dict, temp=15e6, crossSection = 1)->tuple:
    """
    Integrate the system of differential equations.
    
    Parameters
    ----------
    elements : dict
        Dictionary containing the initial values of the initialAbundances.
    temp : float
        The temperature of the system in Kelvin. Default is 15 million Kelvin for the core of the Sun.
    crossSection : float
        The cross section of the reaction. Default is 1 for the time being.
    masses : dict
        Masses of the initialAbundances in the system.
        
    Returns
    -------
    times : ndarray
        Array containing the time steps.
    X : ndarray
        Array containing the values of the initialAbundances.
    """
    
    t = np.linspace(0, 1e17, 1000)
    dT = t[1] - t[0]
    X = []
    X.append(elements)
    XRelative = [{key: elements[key]/elements['H1'] for key in elements.keys()}]
    
    for i in range(1, len(t)):

        newElems = X[i-1].copy()
        
        dCNO1 = CNO1_cycle(newElems, masses, temp, crossSection)
        dCNO2 = CNO2_cycle(newElems, masses, temp, crossSection)
        
        for key in dCNO1.keys():
            newElems[key] = newElems.get(key, 0) + dCNO1[key]*dT
        
        for key in dCNO2.keys():
            newElems[key] = newElems.get(key, 0) + dCNO2[key]*dT
        
        currentRelative = {}
        for key in newElems.keys():
            currentRelative[key] = newElems[key]/newElems['H1']
        
        X.append(newElems)
        XRelative.append(currentRelative)
    
    return t, X, XRelative

In [ ]:
elementMasses = {
    'H1': 1.007825,
    'C12': 12.000000,
    'C13': 13.003355,
    'N14': 14.003074,
    'N15': 15.000109,
    'O16': 15.994915,
    'F17': 17.002095,
    'O17': 16.999132,
    'alpha': 4.002603,
    'proton': 1.007276
} # in atomic mass units


H1 = 6e26 # Initial abundance of Hydrogen in the Sun

initialAbundances = {
    'H1': H1,
    'He4': 0,
    'C12': H1*0.1,
    'C13': 0,
    'N14': 0,
    'N15': 0,
    'O16': 0,
    'O17': 0
}


In [ ]:
# plot the results
times, X, XRelative = integrate_system(initialAbundances, elementMasses)

plt.figure(figsize=(10, 6))
plt.plot(times, [x['H1'] for x in X], label='H1')
plt.plot(times, [x['C12'] for x in X], label='C12')
plt.plot(times, [x['C13'] for x in X], label='C13')
plt.plot(times, [x['N14'] for x in X], label='N14')
plt.plot(times, [x['N15'] for x in X], label='N15')
plt.plot(times, [x['O16'] for x in X], label='O16')
plt.plot(times, [x['O17'] for x in X], label='O17')
plt.plot(times, [x['He4'] for x in X], label='He4')
plt.yscale('log')
plt.xlabel('Time (s)')
plt.ylabel('Abundance')
plt.legend()
plt.grid()
plt.show()


In [ ]:
print(f"Final abundances: {X[-1]}")

In [ ]:
# Relative abundances of the elements compared to H1 at all times

plt.Figure(figsize=(10, 6))
plt.plot(times, [x['H1'] for x in XRelative], label='H1')
plt.plot(times, [x['C12'] for x in XRelative], label='C12')
plt.plot(times, [x['C13'] for x in XRelative], label='C13')
plt.plot(times, [x['N14'] for x in XRelative], label='N14')
plt.plot(times, [x['N15'] for x in XRelative], label='N15')
plt.plot(times, [x['O16'] for x in XRelative], label='O16')
plt.plot(times, [x['O17'] for x in XRelative], label='O17')
plt.plot(times, [x['He4'] for x in XRelative], label='He4')
plt.yscale('log')
plt.xlabel('Time (s)')
plt.ylabel('Relative Abundance')
plt.legend()
plt.grid()
plt.show()
